# Crypto API Project

## Part A: API Integration with CoinMarketCap

This section demonstrates how to retrieve the latest cryptocurrency listings using the CoinMarketCap API.
The Python code below is used to send a request to the CoinMarketCap API and handle the response.

### Setup

- First, we need to import the necessary modules. We'll use the `requests` library to handle HTTP requests and the `json` module to parse the API response.
 - Define the parameters start (The starting point for the cryptocurrency listings), limit (The number of listings to retrieve), convert (The currency in which to display the prices
 - We use a Session object to manage the request. The session's headers are updated with the authentication details.
 - The request is sent using the get method, and the response is processed. The response is converted from JSON format to a Python dictionary.

> **Note:** The API requires an API key for authentication. This key is included in the request headers.




In [ ]:
# API which was taken from the website CoinMarketCap

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15', # the limitation corresponding to the rows
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '496c8a65-36d9-4784-8361-86d603e64dc2',  # copy paste key from the api website
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)

In [ ]:
# Import library pandas
import pandas as pd


#This allows us to see all the columns of the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#This normalizes the data and makes it all pretty in a dataframe
#The below code will be included in the function

df = pd.json_normalize(data['data']) # it will convert the nested JSON into a flat DataFrame
df['timestamp'] = pd.to_datetime('now') # add column at the end for the currently time
df

# Part B: Function api_runner

In this section, we continue from the API request made in the previous part. We process the retrieved data, add a timestamp, and manage the CSV file where the data will be saved, all in one function.

### Data Normalization

After retrieving the data from the CoinMarketCap API, we need to transform it into a format suitable for analysis. This involves converting the JSON response into a pandas DataFrame.

### Adding a Timestamp

To track when the data was retrieved, we add a timestamp column to the DataFrame.

### Saving Data to a CSV File

The DataFrame is then saved to a CSV file. This involves checking if the file already exists and deciding whether to create a new file or append to an existing one.


In [ ]:
# Fumction
# csv file creation

def api_runner():

    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15', # the limitation corresponding to the rows
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '496c8a65-36d9-4784-8361-86d603e64dc2',  # copy paste key from the api website
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)


    df = pd.json_normalize(data['data'])

    # adds an extar column in the dataframe with datetime
    df['timestamp'] = pd.to_datetime('now')
    df

    if not os.path.isfile(r'C:\Users\User\Python Scripts\API.csv'):
        df.to_csv(r'C:\Users\User\Python Scripts\API.csv', header = 'column_names')
    else:
        df.to_csv(r'C:\Users\User\Python Scripts\API.csv', mode = 'a', header = False) # it will append the data and not overwrite them 
    

# Part C: Executing the Function with a Timer

In this section, we demonstrate how to repeatedly execute the `api_runner()` function at regular intervals. We use the `time` library to introduce a delay between each execution, ensuring that the function runs periodically without manual intervention.




In [ ]:
# call the functiom
# import time library to set a timer 

import os
from time import time
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner completed')
    sleep(60) # sleep for 1 minute
exit()

# Part D: Data Analysis and Transformation

In this section, we process the CSV file created earlier to perform various data transformations and calculations. This involves reading the data, setting display options, and performing operations to analyze percentage changes in cryptocurrency values.

### Reading the CSV File

First, we read the data from the CSV file into a pandas DataFrame.


In [ ]:
# read the file - collect the data

df_2 = pd.read_csv(r'C:\Users\User\Python Scripts\API.csv')
df_2.head(10)

### Setting Float Display Format

To improve the readability of floating-point numbers, we set a default format for displaying float values.

In [ ]:
# set a default float format for each value

pd.set_option('display.float_format', lambda x: '%.5f' %x)

### Calculating Percentage Changes

We calculate the average percentage changes for each cryptocurrency over different time periods.


In [ ]:
# create a new dataframe which contains data about the percentage chaneges of each crypto currency

df3 = df.groupby('name',sort = False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d']].mean()
df3

### Transforming DataFrames

We transform the DataFrame to make the data more suitable for analysis and visualization

In [ ]:
# now its not longer a data frame
# we do it to modify the format of the dataframe

df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
# we convert it now to a frame as we want it
df5 = df4.to_frame(name = 'values')
df5

In [ ]:
# quantity of values and datatype
df5.count()

In [ ]:
# name is acting like an index which we dont want beacause we want to be able to use it

index = pd.Index(range(60))

# df6 = df5.set_index(index) # first we did that
df6 = df5.reset_index()
df6

In [ ]:
# Rename Columns
df7 = df6.rename(columns = {'level_1': 'percent_change'})
df7

### Renaming Values for Visualization

Finally, we replace column names with more descriptive labels to aid in visualization.



In [ ]:
# Rename the specific rows which help us for the visualization
replace_dict = {
    'quote.USD.percent_change_1h': '1h',
    'quote.USD.percent_change_24h': '24h',
    'quote.USD.percent_change_7d': '7d',
    'quote.USD.percent_change_30d': '30d'}
df7['percent_change'] = df7['percent_change'].replace(replace_dict)
df7

# Part E: Data Visualization

In this section, we use Seaborn and Matplotlib to visualize the percentage changes in cryptocurrency values. Data visualization helps in understanding patterns and trends effectively.


In [ ]:
# import library related to visualizations
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x = 'percent_change', y = 'values', hue = 'name', data = df7, kind = 'point')

#width = 50
#height = 100
#sns.set(rc={'figure.figsize':(width, height)})

In [ ]:
# querry method to print out a specific row based on determined parameters
df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10